In [1]:
import pandas as pd
from statsmodels.stats.multitest import fdrcorrection as fdr



In [23]:
methods = ["FiLM", "TAG", "GCN", "N2V+MLP", "MLP"]
diseases = ["Cardiovascular Disease", "Immune Dysregulation", "Body Mass Disorders", "Insulin Disorders", "Diabetes"]
outer_dfs = []
global_pvals = []
for value in diseases:
    #results_df = pd.DataFrame(columns=["Mendelian/Method", "CS", "log odds", "a", "b", "c", "d", "se", "delta_log_odds (C-M)", "se_delta", "Z", "pval unadjusted", "pval adjusted (FDR)", "result"])
    df = pd.read_csv("../statistical_dump/{}_drugtarget_unbiased.tsv".format(value), sep="\t", header=0).iloc[:-1 , :]
    outer_dfs.append(df)
    global_pvals.extend(df["pval xDC unadjusted"])

global_pvals_adjusted = fdr(global_pvals)[1]


In [24]:
import numpy as np

print((np.asarray(global_pvals) < 0.05).sum() )
print((np.asarray(global_pvals_adjusted) < 0.05).sum())

89
89


In [25]:
for value, df, df_adj_pvals in zip(diseases, outer_dfs, np.array_split(global_pvals_adjusted, len(outer_dfs))):
    #results_df = pd.DataFrame(columns=["Mendelian/Method", "CS", "log odds", "a", "b", "c", "d", "se", "delta_log_odds (C-M)", "se_delta", "Z", "pval unadjusted", "pval adjusted (FDR)", "result"])
    df["pval xDC adjusted (FDR)"] = df_adj_pvals
    df.to_csv("../statistical_dump/{}_drugtarget_unbiased.tsv".format(value), sep="\t", index=False)